In [1]:
!pip install sentence-transformers datasets

In [2]:
from datasets import load_dataset 
imdb = load_dataset("imdb")

Reusing dataset imdb (/root/.cache/huggingface/datasets/imdb/plain_text/1.0.0/e3c66f1788a67a89c7058d97ff62b6c30531e05b549de56d3ab91891f0561f9a)


In [3]:
imdb = imdb.shuffle()

Loading cached shuffled indices for dataset at /root/.cache/huggingface/datasets/imdb/plain_text/1.0.0/e3c66f1788a67a89c7058d97ff62b6c30531e05b549de56d3ab91891f0561f9a/cache-3baf76060b190d76.arrow


In [4]:
imdb_x = [x for x in imdb['train'][:1000]['text']] 
labels = [x for x in imdb['train'][:1000]['label']] 
import pandas as pd 
pd.DataFrame(imdb_x, 
             columns=["text"]).to_excel( 
                                 "imdb.xlsx", 
                                  index=None) 

In [5]:
pd.read_excel("KHMER.xlsx") 

,text
0,ខណៈពេលដែលជំនាន់ដើម ១៩៣២ ជាមួយប្រេសស្តុនហ្វតគឺល...
1,ភាពយន្តអាមេរិចដែលនិយាយច្រើនបំផុតនៅក្នុងមហោស្រព...
2,ប៊ីងហ្គោគឺជាហ្គេមល្បែងអុកគឺជាឈ្មោះ។ កម្រមានអេក...
3,មានហេតុផលជាច្រើនដែលធ្វើឱ្យខ្ញុំវាយតម្លៃសោណុសយ៉...
4,ខ្ញុំគិតថាការមើលវីដេអូការងារស្តីពីការអនុលោមតាម...
...,...
995,ខ្ញុំបានឃើញខ្សែភាពយន្តនេះជាលើកដំបូងនៅទូរទស្សន៍...
996,ខ្សែភាពយន្តនេះល្អណាស់។ ខ្ញុំមិនមែនជាអ្នកគាំទ្រ...
997,ផលប៉ះពាល់ភ្លាមៗគឺជាខ្សែភាពយន្តទី ៤ នៃខ្សែភាពយន...
998,កាលពី ៣ ឆ្នាំមុនរ៉ាជែល (ថេរ៉េសហ្វ្រែលវែល) កំពុ...


In [6]:
imdb_khmer = list(pd.read_excel("KHMER.xlsx").text) 

In [7]:
from sklearn.model_selection import train_test_split

In [8]:
train_x, test_x, train_y, test_y, khmer_train, khmer_test = train_test_split(imdb_x, labels, imdb_khmer, test_size = 0.2, random_state = 1) 

In [9]:
from sentence_transformers import SentenceTransformer 
model = SentenceTransformer("stsb-xlm-r-multilingual") 

In [10]:
encoded_train = model.encode(train_x) 
encoded_test = model.encode(test_x) 
encoded_khmer_test = model.encode(khmer_test) 

In [11]:
import numpy as np 
train_y = np.array(train_y) 
test_y = np.array(test_y) 

In [12]:
import tensorflow as tf 
input_ = tf.keras.layers.Input((768,)) 
classification = tf.keras.layers.Dense( 
                                1, 
                                activation="sigmoid")(input_) 
classification_model = tf.keras.Model(input_, classification) 
classification_model.compile( 
               loss=tf.keras.losses.BinaryCrossentropy(), 
               optimizer="Adam", 
               metrics=["accuracy", "Precision", "Recall"]) 

In [18]:
classification_model.fit( 
                     x = encoded_train, 
                     y = train_y, 
                     validation_data=(encoded_test, test_y), 
                     epochs = 20) 

Epoch 1/10
25/25 [==============================] - 0s 6ms/step - loss: 0.4273 - accuracy: 0.8125 - precision: 0.8165 - recall: 0.8282 - val_loss: 0.4970 - val_accuracy: 0.7300 - val_precision: 0.7670 - val_recall: 0.7248
Epoch 2/10
25/25 [==============================] - 0s 5ms/step - loss: 0.4214 - accuracy: 0.8125 - precision: 0.8225 - recall: 0.8186 - val_loss: 0.4947 - val_accuracy: 0.7600 - val_precision: 0.7748 - val_recall: 0.7890
Epoch 3/10
25/25 [==============================] - 0s 7ms/step - loss: 0.4167 - accuracy: 0.8175 - precision: 0.8167 - recall: 0.8401 - val_loss: 0.5008 - val_accuracy: 0.7200 - val_precision: 0.7573 - val_recall: 0.7156
Epoch 4/10
25/25 [==============================] - 0s 5ms/step - loss: 0.4127 - accuracy: 0.8238 - precision: 0.8341 - recall: 0.8282 - val_loss: 0.5013 - val_accuracy: 0.7300 - val_precision: 0.7619 - val_recall: 0.7339
Epoch 5/10
25/25 [==============================] - 0s 5ms/step - loss: 0.4098 - accuracy: 0.8250 - precision: 0

In [19]:
classification_model.evaluate(x = encoded_khmer_test,
                              y = test_y) 

7/7 [==============================] - 0s 4ms/step - loss: 0.5951 - accuracy: 0.7350 - precision: 0.7121 - recall: 0.8624


[0.5950571298599243,
 0.7350000143051147,
 0.7121211886405945,
 0.8623853325843811]